In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Download and unzip (2.2GB)
# !wget http://data.csail.mit.edu/places/places205/testSetPlaces205_resize.tar.gz
# !mkdir images/
# !tar -xzf drive/MyDrive/testSetPlaces205_resize.tar.gz -C 'images/'

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms

from tqdm import tqdm
from network import ColorizeNet
from utils import GrayscaleImageFolder

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip()
])

train_set = GrayscaleImageFolder('images/', transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True, num_workers=4)

In [6]:
criterion = nn.MSELoss()

model = ColorizeNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
# scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[8, 24], gamma=1/3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=1/3, patience=5, verbose=True)

In [7]:
try:
    checkpoint = torch.load('drive/MyDrive/checkpoint_0.001.pth', map_location=device)
    start_epoch = checkpoint['next_epoch']
    model.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    scheduler.load_state_dict(checkpoint['scheduler'])
    
except:
    start_epoch = 0

In [8]:
num_epochs = 64
for epoch in range(start_epoch, num_epochs):
    train_loss = 0
    loop = tqdm(train_loader)
    for batch in loop:
        in_gray, in_ab = batch[0].to(device), batch[1].to(device)
        out_ab = model(in_gray)
        loss = criterion(out_ab, in_ab)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()*in_gray.size(0)
        loop.set_description(f'Epoch [{epoch+1:2d}/{num_epochs}]')
        loop.set_postfix(loss=train_loss)

    scheduler.step(train_loss)
    checkpoint = {
        'next_epoch': epoch + 1,
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'scheduler': scheduler.state_dict()
    }
    torch.save(checkpoint, f'drive/MyDrive/checkpoint_{scheduler._last_lr[0]}.pth')

Epoch [64/64]: 100%|██████████| 321/321 [12:10<00:00,  2.27s/it, loss=95]


In [ ]:
torch.save(model.state_dict(), f'./models/model.pth')  # save the model separately from the checkpoint file